<a href="https://colab.research.google.com/github/subhashpolisetti/MQTT-Reservoir-Data-Pipeline/blob/main/CA_Reservoir_Water_Level_Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving Oroville_WML(Sample),.csv to Oroville_WML(Sample),.csv
Saving Shasta_WML(Sample),.csv to Shasta_WML(Sample),.csv
Saving Sonoma_WML(Sample),.csv to Sonoma_WML(Sample),.csv


In [3]:
import pandas as pd

# Load the CSV files
oroville_data = pd.read_csv('Oroville_WML(Sample),.csv')
shasta_data = pd.read_csv('Shasta_WML(Sample),.csv')
sonoma_data = pd.read_csv('Sonoma_WML(Sample),.csv')

# View the first few rows of each reservoir dataset
print(oroville_data.head())
print(shasta_data.head())
print(sonoma_data.head())


        Date   TAF
0  9/29/2024  2000
1  9/30/2024  2001
2  10/1/2024  1999
3  10/2/2024  2000
4  10/5/2024  2000
        Date   TAF
0  9/29/2024  2720
1  9/30/2024  2727
2  10/1/2024  2721
3  10/2/2024  2727
4  10/5/2024  2722
        Date  TAF
0  9/29/2024  192
1  9/30/2024  193
2  10/1/2024  192
3  10/2/2024  194
4  10/5/2024  193


In [4]:
# Convert data to JSON for MQTT messages
oroville_json = oroville_data.to_dict(orient='records')
shasta_json = shasta_data.to_dict(orient='records')
sonoma_json = sonoma_data.to_dict(orient='records')

# Simulate MQTT topics with JSON messages
mqtt_messages = {
    "OROVILLE/WML": oroville_json,
    "SHASTA/WML": shasta_json,
    "SONOMA/WML": sonoma_json
}


In [5]:
from datetime import datetime
from collections import defaultdict

# Function to aggregate data and generate a daily report
def generate_daily_summary(mqtt_data):
    daily_summary = defaultdict(lambda: {'Oroville': None, 'Shasta': None, 'Sonoma': None, 'Total_TAF': 0})

    for reservoir, data in mqtt_data.items():
        reservoir_name = reservoir.split('/')[0].capitalize()  # Extract reservoir name

        for entry in data:
            date = datetime.strptime(entry['Date'], '%m/%d/%Y').strftime('%Y-%m-%d')
            taf_value = entry['TAF']
            daily_summary[date][reservoir_name] = taf_value
            daily_summary[date]['Total_TAF'] += taf_value

    # Convert to list format for display
    report = [{'Date': date, **values} for date, values in sorted(daily_summary.items())]
    return report

# Generate the report
daily_summary_report = generate_daily_summary(mqtt_messages)


In [6]:
# Display the report as a DataFrame
daily_summary_df = pd.DataFrame(daily_summary_report)
print(daily_summary_df)


         Date  Oroville  Shasta  Sonoma  Total_TAF
0  2024-09-29      2000    2720     192       4912
1  2024-09-30      2001    2727     193       4921
2  2024-10-01      1999    2721     192       4912
3  2024-10-02      2000    2727     194       4921
4  2024-10-04      2000    2721     191       4912
5  2024-10-05      2000    2727     193       9835
6  2024-10-06      2000    2726     192       4918
7  2024-10-07      2000    2727     192       4919


In [7]:
# Save as CSV
daily_summary_df.to_csv('Daily_WML_Summary_Report.csv', index=False)

# Save as JSON
daily_summary_df.to_json('Daily_WML_Summary_Report.json', orient='records')
